# Interactive data visualizations



In [19]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import CustomJS, RadioGroup, Select
from datetime import datetime
from dateutil import parser
from netCDF4 import num2date
from siphon.catalog import TDSCatalog

In [20]:
output_notebook()

Loading BokehJS ...

In [21]:
dt_time = lambda x : parser.parse(str(x))

In [22]:
catUrl = "http://140.105.45.139/thredds/catalog/testAll/catalog.xml";
catalog = TDSCatalog(catUrl)
filenames = list(catalog.datasets)

In [23]:
filechoice = Select(value=filenames[-1],options=filenames)
filechoice.js_on_change("value", CustomJS(code="""
    console.log('multi_choice: value=' + this.value, this.toString())"""))
show(filechoice)

In [24]:
remote = catalog.datasets[filechoice.value]
ds = remote.remote_access()
dev = ds.Device
lat = f'{ds.Sensor_Latitude:.3f}'
lon = f'{ds.Sensor_Longitude:.3f}'
hgt = f'{ds.Sensor_Altitude:.1f}'
time = ds.variables['time']
date = [dt_time(x) for x in
        num2date(time[:],units=time.units,calendar=time.calendar)]
plottable = list(x for x in ds.variables if 'time' in ds.variables[x].dimensions)
plottable = list(x for x in plottable if 'time' not in x)

In [25]:
displaynames = list(ds.variables[x].long_name for x in plottable)
selectable = dict(zip(displaynames,plottable))
varsel = RadioGroup(labels=displaynames,active=0)
varsel.js_on_click(CustomJS(code="""
    console.log('varsel: active=' + this.active, this.toString())
"""))

show(varsel)

In [26]:
var = ds.variables[plottable[varsel.active]]
data = var[:]

In [27]:
plt = figure(x_axis_type="datetime", width=800, tools="", toolbar_location=None)
plt.line(date,data,legend_label=var.long_name)
title = (dev+' '+var.long_name + ' ['+ var.units+']\n'+
         '[lat: '+ lat+', lon: '+lon+', elv: '+hgt+']')
plt.title.text = title
plt.xaxis.axis_label = time.long_name
plt.yaxis.axis_label = var.long_name+ ' ['+ var.units+']'
plt.axis.axis_label_text_font_style = "bold"
show(plt)